In [1]:
import os

In [ ]:
from huggingface_hub import login
login('YOUR LOGIN KEY')

In [3]:
import nltk
from sentence_transformers import SentenceTransformer
import pinecone
import spacy

In [6]:
import re

# Define parameters
chunk_size = 10 # Number of sentences per chunk
overlap_percent = 0.2  # 20% overlap
overlap_size = int(chunk_size * overlap_percent)

# Split the text into sentences
sentences = re.split(r'(?<=[.!?]) +', text)

# Generate chunks with overlap
chunks = []
start = 0
while start < len(sentences):
    end = start + chunk_size
    chunk = " ".join(sentences[start:end])
    chunks.append(chunk)
    start += chunk_size - overlap_size  # Move start by chunk size minus overlap

# Output chunk count and example chunk
print(f"Total chunks created: {len(chunks)}")
print("Example chunk:", chunks[0])



NameError: name 'text' is not defined

In [4]:
# Download sentence tokenizer data if not already installed
nltk.download('punkt')

# Initialize SpaCy for NLP
nlp = spacy.blank("en")

# Load a sentence embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [41]:
# Initialize an empty list to store embeddings
embeddings = []

# Generate embeddings for each chunk and store them in the list
for i, chunk in enumerate(chunks):
    # Generate the embedding vector for each chunk
    embedding = embedder.encode(chunk).tolist()
    embeddings.append(embedding)

# Now `embeddings` is a list of all chunk embeddings
print(f"Total embeddings created: {len(embeddings)}")



Total embeddings created: 10


In [ ]:
from pinecone import Pinecone, ServerlessSpec

api_key = 'YOUR API KEY'  # Replace with your actual API key
pinecone_client = Pinecone(api_key=api_key)

# Create or connect to a specific index
index_name = 'vector-nitor'  # Replace with your desired index name
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=384,  # 384 is the dimension for 'all-MiniLM-L6-v2' embeddings
        metric='cosine',  # Choose an appropriate metric for similarity
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Adjust the region as needed
        )
    )

index = pinecone_client.Index(index_name)


In [47]:
vectors = [(str(i), embeddings[i], {"text": chunks[i]}) for i in range(len(embeddings))]

# Insert data into Pinecone
index.upsert(vectors)

print("Data ingestion into Pinecone complete.")

Data ingestion into Pinecone complete.
